This notebook is a basic example for applying salting method to study ambience interference.

In [ ]:
import saltax
import straxen
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from tabulate import tabulate

straxen.print_versions()

# Data Loading

In [ ]:
# available runs for AmBe and Rn220 respectively
am_runs = ['020298', '020303', '020304']
rn_runs = ['022510','022735','022723','022717']

In [ ]:
# Contextxs loading
st_salt = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='salt',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')
st_simu = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='simu',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')
st_data = saltax.contexts.sxenonnt(runid=21952,
             saltax_mode='data',
             output_folder='/project/lgrandi/yuanlq/salt',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')

In [ ]:
def load_events(runs):
    for i, run in enumerate(runs):
        events_simu_i = st_simu.get_array(run, ('event_info', 'cuts_basic'))
        events_salt_i = st_salt.get_array(run, ('event_info', 'cuts_basic'))
        events_data_i = st_data.get_array(run, ('event_info'))
        truth_i = st_simu.get_array(run, 'truth')
        match_i = st_simu.get_array(run, 'match_acceptance_extended')
        events_salt_matched_to_simu_i, \
            events_simu_matched_to_salt_i = saltax.match(truth_i[(truth_i['z']<-13)&(truth_i['z']>-145)&(truth_i['x']**2+truth_i['y']**2<64**2)], 
                                                         match_i[(truth_i['z']<-13)&(truth_i['z']>-145)&(truth_i['x']**2+truth_i['y']**2<64**2)],
                                                         events_simu_i[(events_simu_i['z']<-13)&(events_simu_i['z']>-145)&(events_simu_i['r']<64)], 
                                                         events_salt_i[(events_salt_i['z']<-13)&(events_salt_i['z']>-145)])    
        if i==0:
            events_simu = events_simu_i
            events_salt = events_salt_i
            events_data = events_data_i
            truth = truth_i
            match = match_i
            events_salt_matched_to_simu = events_salt_matched_to_simu_i
            events_simu_matched_to_salt = events_simu_matched_to_salt_i
        else:
            events_simu = np.concatenate((events_simu,events_simu_i))
            events_salt = np.concatenate((events_salt,events_salt_i))
            events_data = np.concatenate((events_data,events_data_i))
            truth = np.concatenate((truth,truth_i))
            match = np.concatenate((match,match_i))
            events_salt_matched_to_simu = np.concatenate((events_salt_matched_to_simu,
                                                          events_salt_matched_to_simu_i))
            events_simu_matched_to_salt = np.concatenate((events_simu_matched_to_salt,
                                                          events_simu_matched_to_salt_i))
    
    return events_simu, events_salt, events_data, truth, match, events_salt_matched_to_simu, events_simu_matched_to_salt

In [ ]:
# Load Rn220 salted runs
(events_simu, 
 events_salt, 
 events_data, 
 truth, 
 match, 
 events_salt_matched_to_simu, 
 events_simu_matched_to_salt) = load_events(rn_runs)
#np.save('/project/lgrandi/yuanlq/salted/matched_salt_rn220.npy', events_salt_matched_to_simu)
#np.save('/project/lgrandi/yuanlq/salted/matched_simu_rn220.npy', events_simu_matched_to_salt)

#import pandas as pd
#df = pd.DataFrame(events_salt_matched_to_simu)
#column_names = [name for name in events_salt_matched_to_simu.dtype.names]
#df.columns = column_names
#df.to_csv('/project/lgrandi/yuanlq/salted/matched_salt_rn220.csv')

#df = pd.DataFrame(events_simu_matched_to_salt)
#column_names = [name for name in events_salt_matched_to_simu.dtype.names]
#df.columns = column_names
#df.to_csv('/project/lgrandi/yuanlq/salted/matched_simu_rn220.csv')

In [ ]:
(events_simua, 
 events_salta, 
 events_dataa, 
 trutha, 
 matcha, 
 events_salt_matched_to_simua, 
 events_simu_matched_to_salta) = load_events(am_runs)

# Template

In [ ]:
def compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                      n_bins = 31, title='Ambience Interference in SR0 AmBe'):
    cs1_bins = np.linspace(0,100,n_bins)
    salt_med = []
    simu_med = []
    salt_1sig_u = []
    simu_1sig_u = []
    salt_1sig_l = []
    simu_1sig_l = []
    salt_2sig_u = []
    simu_2sig_u = []
    salt_2sig_l = []
    simu_2sig_l = []
    
    for i in range(n_bins-1):
        selected_salt = events_salt_matched_to_simu[(events_salt_matched_to_simu['cs1']>=cs1_bins[i])*
                                                    (events_salt_matched_to_simu['cs1']<cs1_bins[i+1])]
        salt_med.append(np.median(selected_salt['cs2'][selected_salt['cs2']>0]))
        salt_1sig_l.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 16.5))
        salt_1sig_u.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 83.5))
        salt_2sig_l.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 2.5))
        salt_2sig_u.append(np.percentile(selected_salt['cs2'][selected_salt['cs2']>0], 97.5))
        
        selected_simu = events_simu_matched_to_salt[(events_simu_matched_to_salt['cs1']>=cs1_bins[i])*
                                                    (events_simu_matched_to_salt['cs1']<cs1_bins[i+1])]
        simu_med.append(np.median(selected_simu['cs2'][selected_simu['cs2']>0]))
        simu_1sig_l.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 16.5))
        simu_1sig_u.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 83.5))
        simu_2sig_l.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 2.5))
        simu_2sig_u.append(np.percentile(selected_simu['cs2'][selected_simu['cs2']>0], 97.5))
    
    cs1_coord = np.linspace(0,100,n_bins-1) + 100/2/(n_bins-1)
    
    plt.figure(dpi=150)
    plt.scatter(events_salt_matched_to_simu['cs1'],events_salt_matched_to_simu['cs2'],s=0.5, label='Salted', alpha=0.1)
    plt.scatter(events_simu_matched_to_salt['cs1'],events_simu_matched_to_salt['cs2'],s=0.5, label='Simulated', alpha=0.1)
    plt.plot(cs1_coord, salt_med, color='b', label='Salt Median')
    plt.plot(cs1_coord, simu_med, color='r', label='Simu Median')
    plt.plot(cs1_coord, salt_1sig_l, color='b', label='Salt 1sig',linestyle='dashed')
    plt.plot(cs1_coord, salt_1sig_u, color='b', linestyle='dashed')
    plt.plot(cs1_coord, salt_2sig_l, color='b', label='Salt 2sig',linestyle='dashed', alpha=0.5)
    plt.plot(cs1_coord, salt_2sig_u, color='b', linestyle='dashed', alpha=0.5)
    plt.plot(cs1_coord, simu_1sig_l, color='r', label='Simu 1sig',linestyle='dashed')
    plt.plot(cs1_coord, simu_1sig_u, color='r', linestyle='dashed')
    plt.plot(cs1_coord, simu_2sig_l, color='r', label='Simu 2sig',linestyle='dashed', alpha=0.5)
    plt.plot(cs1_coord, simu_2sig_u, color='r', linestyle='dashed', alpha=0.5)
    
    
    plt.legend()
    plt.xlim(0,100)
    plt.ylim(0,6000)
    plt.xlabel('CS1 [PE]')
    plt.ylabel('CS2 [PE]')
    plt.title(title)

In [ ]:
compare_templates(events_salt_matched_to_simua, events_simu_matched_to_salta, 
                      n_bins = 31, title='Ambience Interference in SR0 AmBe')

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                      n_bins = 31, title='Ambience Interference in SR0 Rn220')

## Cuts accpetance

In [ ]:
ALL_CUTS = np.array([
            'cut_ambience',
            'cut_cs2_area_fraction_top',
            'cut_daq_veto',
            'cut_interaction_exists',
            'cut_main_is_valid_triggering_peak',
            'cut_run_boundaries',
            'cut_s1_area_fraction_top',
            'cut_s1_max_pmt',
            'cut_s1_pattern_bottom',
            'cut_s1_pattern_top',
            'cut_s1_single_scatter',
            'cut_s1_tightcoin_3fold',
            'cut_s1_width',
            'cut_s2_pattern',
            'cut_s2_recon_pos_diff',
            'cut_s2_single_scatter',
            'cut_s2_width',
            'cut_shadow'])
ALL_CUTS_EXCEPT_S2Pattern = np.array([
            'cut_ambience',
            'cut_cs2_area_fraction_top',
            'cut_daq_veto',
            'cut_interaction_exists',
            'cut_main_is_valid_triggering_peak',
            'cut_run_boundaries',
            'cut_s1_area_fraction_top',
            'cut_s1_max_pmt',
            'cut_s1_pattern_bottom',
            'cut_s1_pattern_top',
            'cut_s1_single_scatter',
            'cut_s1_tightcoin_3fold',
            'cut_s1_width',
            'cut_s2_recon_pos_diff',
            'cut_s2_single_scatter',
            'cut_s2_width',
            'cut_shadow'])

ALL_CUTS_EXCEPT_S2PatternS1Width = np.array([
            'cut_ambience',
            'cut_cs2_area_fraction_top',
            'cut_daq_veto',
            'cut_interaction_exists',
            'cut_main_is_valid_triggering_peak',
            'cut_run_boundaries',
            'cut_s1_area_fraction_top',
            'cut_s1_max_pmt',
            'cut_s1_pattern_bottom',
            'cut_s1_pattern_top',
            'cut_s1_single_scatter',
            'cut_s1_tightcoin_3fold',
            'cut_s2_recon_pos_diff',
            'cut_s2_single_scatter',
            'cut_s2_width',
            'cut_shadow'])

def apply_n_minus_1_cuts(events_with_cuts, cut_oi, 
                         all_cuts = ALL_CUTS_EXCEPT_S2PatternS1Width):
    other_cuts = all_cuts[all_cuts!=cut_oi]
    mask = np.ones(len(events_with_cuts), dtype=bool)
    
    for cut in other_cuts:
        mask &= events_with_cuts[cut]
    
    return mask

In [ ]:
mask_salt_all_cuts = events_salt_matched_to_simu['cuts_basic']
mask_simu_all_cuts = events_simu_matched_to_salt['cuts_basic']

# Initialize a list to store your rows
table_data = []

# Loop over each cut and calculate the acceptance values
for cut_oi in ALL_CUTS:
    mask_salt_except_cut_oi = apply_n_minus_1_cuts(events_salt_matched_to_simu, cut_oi, 
                                                   all_cuts = ALL_CUTS)
    mask_simu_except_cut_oi = apply_n_minus_1_cuts(events_simu_matched_to_salt, cut_oi, 
                                                   all_cuts = ALL_CUTS)
    acceptance_salt = int(np.sum(mask_salt_all_cuts)/np.sum(mask_salt_except_cut_oi)*100)/100
    acceptance_simu = int(np.sum(mask_simu_all_cuts)/np.sum(mask_simu_except_cut_oi)*100)/100
    
    # Add a row for each cut
    table_data.append([cut_oi, acceptance_salt, acceptance_simu])

# Define the headers
headers = ['Cut Name', 'Acceptance in Salt', 'Acceptance in Simu']

# Print the table
print(tabulate(table_data, headers=headers, tablefmt='grid'))

We need to take out `cut_s2_pattern` and `cut_s1_width` since we cannot make sense from their low acceptance.

In [ ]:
mask_salt_all_cuts_except_s2patterns1width = apply_n_minus_1_cuts(events_salt_matched_to_simu, 
                                                           'cut_s1_width', 
                                                           all_cuts = ALL_CUTS_EXCEPT_S2Pattern)
mask_simu_all_cuts_except_s2patterns1width = apply_n_minus_1_cuts(events_simu_matched_to_salt, 
                                                           'cut_s1_width', 
                                                           all_cuts = ALL_CUTS_EXCEPT_S2Pattern)

# Initialize a list to store your rows
table_data = []

# Loop over each cut and calculate the acceptance values
for cut_oi in ALL_CUTS:
    mask_salt_except_cut_oi = apply_n_minus_1_cuts(events_salt_matched_to_simu, cut_oi)
    mask_simu_except_cut_oi = apply_n_minus_1_cuts(events_simu_matched_to_salt, cut_oi)
    acceptance_salt = int(np.sum(mask_salt_all_cuts_except_s2patterns1width)/
                          np.sum(mask_salt_except_cut_oi)*100)/100
    acceptance_simu = int(np.sum(mask_simu_all_cuts_except_s2patterns1width)/
                          np.sum(mask_simu_except_cut_oi)*100)/100
    
    # Add a row for each cut
    table_data.append([cut_oi, acceptance_salt, acceptance_simu])

# Define the headers
headers = ['Cut Name', 'Acceptance in Salt', 'Acceptance in Simu']

# Print the table
print(tabulate(table_data, headers=headers, tablefmt='grid'))

In [ ]:
compare_templates(events_salt_matched_to_simu[events_salt_matched_to_simu['cut_shadow']], 
                  events_simu_matched_to_salt[events_salt_matched_to_simu['cut_shadow']], 
                  n_bins = 31, title='(Added cut_shadow) Ambience Interference in SR0 Rn220')

In [ ]:
compare_templates(events_salt_matched_to_simu[events_salt_matched_to_simu['cut_ambience']], 
                  events_simu_matched_to_salt[events_salt_matched_to_simu['cut_ambience']], 
                  n_bins = 31, title='(Added cut_shadow) Ambience Interference in SR0 Rn220')

In [ ]:
compare_templates(events_salt_matched_to_simu[events_salt_matched_to_simu['cut_s2_width']], 
                  events_simu_matched_to_salt[events_salt_matched_to_simu['cut_s2_width']], 
                  n_bins = 31, title='(Added cut_s2_width) Ambience Interference in SR0 Rn220')

In [ ]:
compare_templates(events_salt_matched_to_simu[events_salt_matched_to_simu['cut_s1_single_scatter']], 
                  events_simu_matched_to_salt[events_salt_matched_to_simu['cut_s1_single_scatter']], 
                  n_bins = 31, title='(Added cut_s1_single_scatter) Ambience Interference in SR0 Rn220')

In [ ]:
compare_templates(events_salt_matched_to_simu[events_salt_matched_to_simu['cut_daq_veto']], 
                  events_simu_matched_to_salt[events_salt_matched_to_simu['cut_daq_veto']], 
                  n_bins = 31, title='(Added cut_daq_veto) Ambience Interference in SR0 Rn220')

# Z dependence

In [ ]:
n_bins = 31
z_bins = np.linspace(zmin,zmax,n_bins)
zs = (z_bins[1:]+z_bins[:-1])/2
zmin = np.min(events_salt_matched_to_simu['z'])
zmax = np.max(events_salt_matched_to_simu['z'])

med = []
sig1_l = []
sig1_u = []
sig2_l = []
sig2_u = []

for i in range(n_bins-1):
    selected_i = np.where((events_salt_matched_to_simu['z']>=z_bins[i]) *
                          (events_salt_matched_to_simu['z']<=z_bins[i+1]))
    ds = events_salt_matched_to_simu[selected_i]['cs2'] - events_simu_matched_to_salt[selected_i]['cs2']
    ds = ds[~np.isnan(ds)]
    
    med.append(np.median(ds))
    sig1_l.append(np.percentile(ds, 16.5))
    sig1_u.append(np.percentile(ds, 83.5))
    sig2_l.append(np.percentile(ds, 2.5))
    sig2_u.append(np.percentile(ds, 97.5))

plt.figure(dpi=150)
plt.scatter(events_salt_matched_to_simu['z'], 
            events_salt_matched_to_simu['cs2']-events_simu_matched_to_salt['cs2'], 
            s=1, label='Rn220', alpha=0.2)
plt.plot(zs, med, color='r', label='Median')
plt.plot(zs, sig1_l, color='r', label='1sig',linestyle='dashed')
plt.plot(zs, sig1_u, color='r', linestyle='dashed')
plt.plot(zs, sig2_l, color='r', label='2sig',linestyle='dashed', alpha=0.5)
plt.plot(zs, sig2_u, color='r', linestyle='dashed', alpha=0.5)
plt.ylim(-10,90)
#plt.yscale('log')
plt.legend()
plt.title('Salted Rn220 into SR0 Rn220')
plt.xlabel('Simu Z [cm]')
plt.ylabel('Ambience Contribution in CS2 [PE]')

In [ ]:
n_bins = 31
z_bins = np.linspace(zmin,zmax,n_bins)
zs = (z_bins[1:]+z_bins[:-1])/2
zmin = np.min(events_salt_matched_to_simu['z'])
zmax = np.max(events_salt_matched_to_simu['z'])

med = []
sig1_l = []
sig1_u = []
sig2_l = []
sig2_u = []

for i in range(n_bins-1):
    selected_i = np.where((events_salt_matched_to_simu['z']>=z_bins[i]) *
                          (events_salt_matched_to_simu['z']<=z_bins[i+1]))
    ds = events_salt_matched_to_simu[selected_i]['cs1'] - events_simu_matched_to_salt[selected_i]['cs1']
    ds = ds[~np.isnan(ds)]
    
    med.append(np.median(ds))
    sig1_l.append(np.percentile(ds, 16.5))
    sig1_u.append(np.percentile(ds, 83.5))
    sig2_l.append(np.percentile(ds, 2.5))
    sig2_u.append(np.percentile(ds, 97.5))

plt.figure(dpi=150)
plt.scatter(events_salt_matched_to_simu['z'], 
            events_salt_matched_to_simu['cs1']-events_simu_matched_to_salt['cs1'], 
            s=1, label='Rn220', alpha=0.2)
plt.plot(zs, med, color='r', label='Median')
plt.plot(zs, sig1_l, color='r', label='1sig',linestyle='dashed')
plt.plot(zs, sig1_u, color='r', linestyle='dashed')
plt.plot(zs, sig2_l, color='r', label='2sig',linestyle='dashed', alpha=0.5)
plt.plot(zs, sig2_u, color='r', linestyle='dashed', alpha=0.5)
plt.ylim(-1,5)
#plt.yscale('log')
plt.legend()
plt.title('Salted Rn220 into SR0 Rn220')
plt.xlabel('Simu Z [cm]')
plt.ylabel('Ambience Contribution in CS1 [PE]')

In [ ]:
n_bins = 31
z_bins = np.linspace(zmin,zmax,n_bins)
zs = (z_bins[1:]+z_bins[:-1])/2
zmin = np.min(events_salt_matched_to_simua['z'])
zmax = np.max(events_salt_matched_to_simua['z'])

med = []
sig1_l = []
sig1_u = []
sig2_l = []
sig2_u = []

for i in range(n_bins-1):
    selected_i = np.where((events_salt_matched_to_simua['z']>=z_bins[i]) *
                          (events_salt_matched_to_simua['z']<=z_bins[i+1]))
    ds = events_salt_matched_to_simua[selected_i]['cs2'] - events_simu_matched_to_salta[selected_i]['cs2']
    ds = ds[~np.isnan(ds)]
    
    med.append(np.median(ds))
    sig1_l.append(np.percentile(ds, 16.5))
    sig1_u.append(np.percentile(ds, 83.5))
    sig2_l.append(np.percentile(ds, 2.5))
    sig2_u.append(np.percentile(ds, 97.5))

plt.figure(dpi=150)
plt.scatter(events_salt_matched_to_simua['z'], 
            events_salt_matched_to_simua['cs2']-events_simu_matched_to_salta['cs2'], 
            s=1, label='Rn220', alpha=0.2)
plt.plot(zs, med, color='r', label='Median')
plt.plot(zs, sig1_l, color='r', label='1sig',linestyle='dashed')
plt.plot(zs, sig1_u, color='r', linestyle='dashed')
plt.plot(zs, sig2_l, color='r', label='2sig',linestyle='dashed', alpha=0.5)
plt.plot(zs, sig2_u, color='r', linestyle='dashed', alpha=0.5)
plt.ylim(-10,90)
#plt.yscale('log')
plt.legend()
plt.title('Salted Rn220 into SR0 AmBe')
plt.xlabel('Simu Z [cm]')
plt.ylabel('Ambience Contribution in CS2 [PE]')